### In this file there are three models trained, each with different length of input sequence

In [1]:
from imports import *

2024-06-27 15:31:01.932258: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-27 15:31:01.956850: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-27 15:31:01.956870: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-27 15:31:01.956887: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-27 15:31:01.961789: I tensorflow/core/platform/cpu_feature_g

In [2]:
sys.path.append('src')
from process_data import DataProcessor
from vanilla_transformer import Transformer
from visualize_data import Visualizer
from train_model import ModelTrainer
from load_data import DataLoader
model_trainer = ModelTrainer()
data_processor = DataProcessor()
visualizer = Visualizer()
data_loader = DataLoader()

In [3]:
data, pip_factor = data_loader.load_gpbusd_5min()

In [4]:
train_pct = 0.6
val_pct = 0.2

In [5]:
b, a = butter(5, 1/6, btype='low', analog=False)
data['bw_open'] = filtfilt(b, a, data['open'])
data['bw_high'] = filtfilt(b, a, data['high'])
data['bw_low'] = filtfilt(b, a, data['low'])
data['bw_close'] = filtfilt(b, a, data['close'])

In [6]:
data['rsi_close_5'] = abstract.RSI(data, timeperiod=7, price='bw_close')
data['willr_close_8'] = abstract.WILLR(data, timeperiod=12, price='bw_close')

In [7]:
data.dropna(how='any', axis=0, inplace=True)

In [8]:
data, _ = data_processor.normalize_custom(data, train_pct, ['volume', 'rsi_close_5', 'willr_close_8'])

In [9]:
target_time = data[['time', 'bw_close', 'close']]

In [10]:
data = data_processor.diff_features(data, ['close', 'bw_open', 'bw_high', 'bw_low', 'bw_close'])

In [11]:
data.loc[:, 'close'] *= pip_factor
data.loc[:, 'bw_close'] *= pip_factor
data.loc[:, 'bw_open'] *= pip_factor
data.loc[:, 'bw_high'] *= pip_factor
data.loc[:, 'bw_low'] *= pip_factor

In [12]:
target = data[['close', 'bw_close']]

In [13]:
data.drop(['open', 'high', 'low', 'close'], axis=1, inplace=True)

/tmp/ipykernel_52219/4177561520.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.drop(['open', 'high', 'low', 'close'], axis=1, inplace=True)


In [14]:
input_seq_len_0 = 32
input_seq_len_1 = 16
input_seq_len_2 = 8

output_seq_len = 8

# 32 steps input - 160 minutes

In [42]:
config = {
    'input_seq_len': input_seq_len_0,
    'output_seq_len': output_seq_len,
    'multihead_attn_heads': 4,
    'd_k': 32,
    'd_v': 32,
    'd_ff': 32,
    'd_out': 1,
    'd_model': 32,
    'encoder_layers': 1,
    'decoder_layers': 1,
    'dropout_rate': 0.1,
    'activation': 'relu',
    'batch_size': 32,
}

In [43]:
Xdata, ydata = model_trainer.create_Xy_data(data, target, input_seq_len_0, output_seq_len)

In [44]:
np.random.seed(42)
shuffled_indices = np.random.permutation(Xdata.shape[0])
Xdata = Xdata[shuffled_indices]
ydata = ydata[shuffled_indices]

In [45]:
Xtrain_time, ytrain_c, Xval_time, yval_c, Xtest_time, ytest_c = model_trainer.split_data(Xdata, ydata, train_pct, val_pct)

In [46]:
Xtrain = (Xtrain_time[:, :, 1:]).astype(np.float64)
Xval = (Xval_time[:, :, 1:]).astype(np.float64)
Xtest = (Xtest_time[:, :, 1:]).astype(np.float64)

ytrain = ytrain_c[:, :, 1:]
yval = yval_c[:, :, 1:]
ytest = ytest_c[:, :, 1:]

In [47]:
epochs = 15

path = 'saved_models/tf-5m-32in-8out.h5'

model = model_trainer.load_model_transformer(path, Xtest, config)

#model = Transformer(config)

In [48]:
#model, best_epoch, train_losses, val_losses = model_trainer.train_model_transformer(Xtrain, ytrain, Xval, yval, path, epochs, model_0, 1e-3, config, Xtest)

In [49]:
with tf.device('/CPU:0'):
    pred = model(Xtest, target=None, training=False)

In [50]:
real_price_movement = list(data_processor.sum_along_seq_dim(ytest_c[:,:,0]))
pred_price_movement = list(data_processor.sum_along_seq_dim(pred[:,:,0]))

In [51]:
real_classes = data_processor.create_binary_classification_classes(real_price_movement, 0.0)
pred_classes = data_processor.create_binary_classification_classes(pred_price_movement, 0.0)

In [52]:
accuracy_0, precision_0, recall_0, f1_0 = data_processor.create_metrics_from_classes_binary(real_classes, pred_classes)
visualizer.create_metrics_df(accuracy_0, precision_0, recall_0, f1_0)

,Metric,Value
0,Accuracy,0.782986
1,Precision,0.790971
2,Recall,0.781408
3,F1 Score,0.786160


# 16 steps input - 80 minutes

In [15]:
config = {
    'input_seq_len': input_seq_len_1,
    'output_seq_len': output_seq_len,
    'multihead_attn_heads': 4,
    'd_k': 32,
    'd_v': 32,
    'd_ff': 32,
    'd_out': 1,
    'd_model': 32,
    'encoder_layers': 1,
    'decoder_layers': 1,
    'dropout_rate': 0.1,
    'activation': 'relu',
    'batch_size': 32,
}

In [16]:
Xdata, ydata = model_trainer.create_Xy_data(data, target, input_seq_len_1, output_seq_len)

In [17]:
np.random.seed(42)
shuffled_indices = np.random.permutation(Xdata.shape[0])
Xdata = Xdata[shuffled_indices]
ydata = ydata[shuffled_indices]

In [18]:
Xtrain_time, ytrain_c, Xval_time, yval_c, Xtest_time, ytest_c = model_trainer.split_data(Xdata, ydata, train_pct, val_pct)

In [19]:
Xtrain = (Xtrain_time[:, :, 1:]).astype(np.float64)
Xval = (Xval_time[:, :, 1:]).astype(np.float64)
Xtest = (Xtest_time[:, :, 1:]).astype(np.float64)

ytrain = ytrain_c[:, :, 1:]
yval = yval_c[:, :, 1:]
ytest = ytest_c[:, :, 1:]

In [21]:
epochs = 15

path = 'saved_models/tf-5m-16in-8out.h5'

model = model_trainer.load_model_transformer(path, Xtest, config)

#model = Transformer(config)

In [22]:
#model, best_epoch, train_losses, val_losses = model_trainer.train_model_transformer(Xtrain, ytrain, Xval, yval, path, epochs, model, 1e-3, config, Xtest)

In [23]:
with tf.device('/CPU:0'):
    pred = model(Xtest, target=None, training=False)

In [24]:
real_price_movement = list(data_processor.sum_along_seq_dim(ytest_c[:,:,0]))
pred_price_movement = list(data_processor.sum_along_seq_dim(pred[:,:,0]))

In [25]:
real_classes = data_processor.create_binary_classification_classes(real_price_movement, 0.0)
pred_classes = data_processor.create_binary_classification_classes(pred_price_movement, 0.0)

In [26]:
accuracy_0, precision_0, recall_0, f1_0 = data_processor.create_metrics_from_classes_binary(real_classes, pred_classes)
visualizer.create_metrics_df(accuracy_0, precision_0, recall_0, f1_0)

,Metric,Value
0,Accuracy,0.759943
1,Precision,0.777546
2,Recall,0.736703
3,F1 Score,0.756574


# 8 steps input - 40 minutes

In [32]:
config = {
    'input_seq_len': input_seq_len_2,
    'output_seq_len': output_seq_len,
    'multihead_attn_heads': 4,
    'd_k': 32,
    'd_v': 32,
    'd_ff': 32,
    'd_out': 1,
    'd_model': 32,
    'encoder_layers': 1,
    'decoder_layers': 1,
    'dropout_rate': 0.1,
    'activation': 'relu',
    'batch_size': 32,
}

In [33]:
Xdata, ydata = model_trainer.create_Xy_data(data, target, input_seq_len_2, output_seq_len)

In [34]:
np.random.seed(42)
shuffled_indices = np.random.permutation(Xdata.shape[0])
Xdata = Xdata[shuffled_indices]
ydata = ydata[shuffled_indices]

In [35]:
Xtrain_time, ytrain_c, Xval_time, yval_c, Xtest_time, ytest_c = model_trainer.split_data(Xdata, ydata, train_pct, val_pct)

In [36]:
Xtrain = (Xtrain_time[:, :, 1:]).astype(np.float64)
Xval = (Xval_time[:, :, 1:]).astype(np.float64)
Xtest = (Xtest_time[:, :, 1:]).astype(np.float64)

ytrain = ytrain_c[:, :, 1:]
yval = yval_c[:, :, 1:]
ytest = ytest_c[:, :, 1:]

In [37]:
epochs = 15

path = 'saved_models/tf-5m-8in-8out.h5'

model = model_trainer.load_model_transformer(path, Xtest, config)

#model = Transformer(config)

In [38]:
#model, best_epoch, train_losses, val_losses = model_trainer.train_model_transformer(Xtrain, ytrain, Xval, yval, path, epochs, model, 1e-3, config, Xtest_2)

In [39]:
with tf.device('/CPU:0'):
    pred = model(Xtest, target=None, training=False)

In [40]:
real_price_movement = list(data_processor.sum_along_seq_dim(ytest_c[:,:,0]))
pred_price_movement = list(data_processor.sum_along_seq_dim(pred[:,:,0]))

In [41]:
real_classes = data_processor.create_binary_classification_classes(real_price_movement, 0.0)
pred_classes = data_processor.create_binary_classification_classes(pred_price_movement, 0.0)

In [42]:
accuracy_0, precision_0, recall_0, f1_0 = data_processor.create_metrics_from_classes_binary(real_classes, pred_classes)
visualizer.create_metrics_df(accuracy_0, precision_0, recall_0, f1_0)

,Metric,Value
0,Accuracy,0.764068
1,Precision,0.735524
2,Recall,0.835141
3,F1 Score,0.782174
